In [1]:
# def categorize_dnn_output(df, score_name, channel, region, year):
def categorize_dnn_output(df, score_name, region, year="2018"):
    """
    we only need VBF sample in h-peak region for this. No ggh, no sidebands
    """
    channel="vbf"
    # Run 2 (VBF yields)
    target_yields = {
        "2016": [
            0.35455259,
            0.50239086,
            0.51152889,
            0.52135985,
            0.5282209,
            0.54285134,
            0.54923751,
            0.56504687,
            0.57204477,
            0.58273066,
            0.5862248,
            0.59568793,
            0.60871905,
        ],
        "2017": [
            0.44194544,
            0.55885064,
            0.57796123,
            0.58007343,
            0.58141001,
            0.58144682,
            0.57858609,
            0.59887533,
            0.59511901,
            0.59644831,
            0.59825915,
            0.59283309,
            0.57329743,
        ],
        "2018": [
            0.22036263,
            1.31808978,
            1.25396849,
            1.183724,
            1.12620194,
            1.06041376,
            0.99941623,
            0.93224412,
            0.87074753,
            0.80599462,
            0.73469265,
            0.668018,
            0.60991945,
        ],
    }

    bins = [df[score_name].max()]

    # slicer = (
    #     (df.channel_nominal == channel) & (df.region == region) & (df.year == int(year))
    # )
    slicer = (
        (df.region == region)
    )
    df_sorted = (
        df.loc[slicer, :]
        .sort_values(by=score_name, ascending=False)
        .reset_index(drop=True)
    )
    df_sorted["wgt_cumsum"] = df_sorted.wgt_nominal.cumsum()

    tot_yield = 0
    last_yield = 0
    for yi in reversed(target_yields[year]):
        tot_yield += yi
        print(f"yi: {yi}")
        for i in range(df_sorted.shape[0] - 1):
            value = df_sorted.loc[i, "wgt_cumsum"]
            value1 = df_sorted.loc[i + 1, "wgt_cumsum"]
            if (value < tot_yield) & (value1 > tot_yield):
                if abs(last_yield - tot_yield) < 1e-06:
                    continue
                if abs(tot_yield - sum(target_yields[year])) < 1e-06:
                    continue
                bins.append(df_sorted.loc[i, score_name])
                last_yield = tot_yield
    bins.append(0.0)
    bins = sorted(bins)
    print(bins)

In [17]:
import ROOT as rt

ldd: /depot/cms/kernels/root632/bin/python3.12 (deleted): No such file or directory


In [19]:
file = rt.TFile("/work/users/yun79/valerie/fork/copperheadV2/data/reweight_zpt_2018_nJetBinned_new.histo.root")

In [22]:
# file.ls()
file["Reweight_0jet"].Print("v")

TH1.Print Name  = Reweight_0jet, Entries= 8895, Total sum= 40.1642


In [2]:
import awkward as ak
import dask_awkward as dak
import pandas as pd
import glob

In [9]:
global_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/"
# label = "DmitryMaster_JECoff_GeofitFixed_Nov01"

# label = "rereco_yun_Nov04"
label = "JecOffGfOn_Nov16"


year="2018"
# # full_load_path = f"{global_path}/{label}/stage2_unbinned/vbf_2018/vbf_powheg_dipole*.parquet"
# full_load_path = f"{global_path}/{label}/stage2_unbinned/vbf_{year}/vbf_powheg_dipole*.parquet"
# events = dak.from_parquet(full_load_path)

vbfs = glob.glob(f"{global_path}/{label}/stage2_unbinned/vbf_{year}/vbf_powheg_dipole*.parquet")
# gghs = glob.glob(f"{global_path}/{label}/stage2_unbinned/vbf_{year}/ggh*.parquet")
# full_load_path = 
events = dak.from_parquet(vbfs) # only vbfs if h-peak region is used

In [10]:
label

'JecOffGfOn_Nov16'

In [11]:
# ! ls /depot/cms/users/yun79/hmm/copperheadV1clean/DmitryMaster_JECoff_GeofitFixed_Nov01/stage2_unbinned/vbf_2018

In [12]:
events.fields

['dimuon_mass',
 'event',
 'wgt_nominal',
 'mu1_pt',
 'score_pytorch_jun27_nominal',
 'region',
 'year',
 'entry']

In [13]:
df  = ak.to_dataframe(ak.zip({
    "score_pytorch_jun27_nominal":events.score_pytorch_jun27_nominal,
    "region" :events.region,
    "wgt_nominal": events.wgt_nominal,
})).compute()

In [15]:

categorize_dnn_output(df, "score_pytorch_jun27_nominal", "h-peak", year=year)

yi: 0.60991945
yi: 0.668018
yi: 0.73469265
yi: 0.80599462
yi: 0.87074753
yi: 0.93224412
yi: 0.99941623
yi: 1.06041376
yi: 1.12620194
yi: 1.183724
yi: 1.25396849
yi: 1.31808978
yi: 0.22036263
[0.0, 0.15816980600357056, 0.5751140117645264, 0.8470537662506104, 1.0578068494796753, 1.2360748052597046, 1.3939404487609863, 1.5378074645996094, 1.6691679954528809, 1.7927818298339844, 1.9104204177856445, 2.0226566791534424, 2.1442697048187256, 2.4760870933532715]


In [10]:
yields = [
            0.22036263,
            1.31808978,
            1.25396849,
            1.183724,
            1.12620194,
            1.06041376,
            0.99941623,
            0.93224412,
            0.87074753,
            0.80599462,
            0.73469265,
            0.668018,
            0.60991945,
        ]
sum(yields)

11.7837932

In [4]:
import dask_awkward as dak
load_path = "/depot/cms/users/yun79/hmm/copperheadV1clean/rereco_yun_gfOn_JecOff_Nov06/stage1_output/2017/vbf_powheg_dipole"

events = dak.from_parquet(f"{load_path}/*.parquet")

In [5]:
events.fields

['dataset',
 'dimuon_cos_theta_cs',
 'dimuon_cos_theta_cs_pisa',
 'dimuon_dEta',
 'dimuon_dPhi',
 'dimuon_dR',
 'dimuon_ebe_mass_res',
 'dimuon_ebe_mass_res_rel',
 'dimuon_eta',
 'dimuon_mass',
 'dimuon_phi',
 'dimuon_phi_cs',
 'dimuon_phi_cs_pisa',
 'dimuon_pisa_mass_res',
 'dimuon_pisa_mass_res_rel',
 'dimuon_pt',
 'dimuon_pt_log',
 'event',
 'gjet1_eta',
 'gjet1_mass',
 'gjet1_phi',
 'gjet1_pt',
 'gjet2_eta',
 'gjet2_mass',
 'gjet2_phi',
 'gjet2_pt',
 'gjj_dEta',
 'gjj_dPhi',
 'gjj_dR',
 'gjj_eta',
 'gjj_mass',
 'gjj_phi',
 'gjj_pt',
 'htsoft2_Absolute2017_down',
 'htsoft2_Absolute2017_up',
 'htsoft2_Absolute_down',
 'htsoft2_Absolute_up',
 'htsoft2_BBEC12017_down',
 'htsoft2_BBEC12017_up',
 'htsoft2_BBEC1_down',
 'htsoft2_BBEC1_up',
 'htsoft2_EC22017_down',
 'htsoft2_EC22017_up',
 'htsoft2_EC2_down',
 'htsoft2_EC2_up',
 'htsoft2_FlavorQCD_down',
 'htsoft2_FlavorQCD_up',
 'htsoft2_HF2017_down',
 'htsoft2_HF2017_up',
 'htsoft2_HF_down',
 'htsoft2_HF_up',
 'htsoft2_RelativeBal_down',


In [11]:
# is_h_peak = events.region == "h-peak"
wgt_nominal = events.wgt_nominal.compute()
import awkward as ak
ak.sum(wgt_nominal)

20.15027637320125